In [0]:
!pip install tika
import re
from tika import parser

In [0]:
from google.colab import files
files.upload()

Saving 061600_Sheathing.pdf to 061600_Sheathing.pdf


{'061600_Sheathing.pdf': b'%PDF-1.7\r%\xe2\xe3\xcf\xd3\r\n1317 0 obj\r<</Filter/FlateDecode/First 154/Length 1252/N 18/Type/ObjStm>>stream\r\nh\xde\xbcVmo\xdb6\x10\xfe+\xfc\xb4&\x18,\xbeSb\xd1fs\xe2\xb8\r\xd0d^\xe2-\xc5\x02\x7f`d\xc6\xd1&K\x82$\'\xcd\xbf\xdf\x1d-9\xae\xd3\xb4i1\x0c\x82\xa4#\xefx|\xee\x8e|H\xab\x18a\xc4\xca\x84p\xad\x88U\x92H\x8b\x7fN\xb4\x96\xd0\xcfIl\x13\xf8\x8b\xa0\xb3R\x12\xce\x19\x0eP\x84\x0bm@\xd0\x84\xcbD\x83\xda\x80\x0b\x03\xb6\nT1\xe7\xa0\xb2\x84\xa3\'\xab\x04\x11"\x81QJ\x13\xa1-\xaa\x18\n8\xdc\x10a\x04\xf6\xc4D\xd8\x18\xa6\x10\x96Hfb\xf2\xe6\r=t\x8d\x1f\x97EK\x0f\x8fF\xc7\xc7\xc7?O\xb3\xa5o\xce\xfc\xfdy\xb9t\xc5\xe4\xe2tJ\x8f\x8b\xb4\x9cg\xc5\x82^f\xc5\xb0h\xb2M{\x9c\xd5M{t\xebj"\x05E\x1f#\xdf\xa4uV\xb5e\x1d\xa2e\xe4\x9c~p\x9d\t\x17\x82^\xac\xae\xdb\x87\xca\xd3i\xbd\xf2\xd3 \xe0\'\xcc~\x99\xcd\xdb\xdb&\xc4\x03\xc3\x0e\x0e\x00\xd9\xb0I}\xd1\x92\xc4r:\xbc[\x04\x03\xa2\x18\xa7G\xaez\xef\xb3\xc5mK\x8c\x95\x14\xe7D\xb3\x81\xe0\x86\x8es\xb7hz4\x87\x87\xe5\xa7\xab\x0

In [0]:
raw = parser.from_file('061600_Sheathing.pdf')
text = raw['content']
print(text)

In [0]:
print(text.strip())

In [0]:
#pattern = re.compile("SECTION[ ]?[0-9].*[ ]?-.*")
p_section = re.compile("SECTION \d{6} - [A-Z]*")
sections = re.findall(p_section, text)
if (len(sections) > 0):
    section = sections[-1]
    print(section)
else:
    print("Section cannot be found")

section_num = section.split()[1]
section_name = section.split()[-1]
print("Section Number: ", section_num)
print("Section Name: ", section_name)

SECTION 061600 - SHEATHING
Section Number:  061600
Section Name:  SHEATHING


In [0]:
pattern = re.compile("PART 2 - [a-zA-Z]*")
str2 = re.findall(pattern, text)
print(str2[0])

PART 2 - PRODUCTS


In [0]:
text = text[text.find(str2[0]):] #index of str2[0] to the end of the text
print(text)

In [0]:
p_category = re.compile("2[.][0-9] [A-Z -]*[\n]")
p_category2 = re.compile("2[.][0-9][0-9] [A-Z -]*[\n]")
categories = re.findall(p_category, text)
categories2 = re.findall(p_category2, text)
print(categories2)
for i in categories2:
  categories.append(i)
print (categories)

['2.10 FASTENERS \n', '2.11 SHEATHING JOINT-AND-PENETRATION TREATMENT MATERIALS \n', '2.12 MISCELLANEOUS MATERIALS \n']
['2.1 PERFORMANCE REQUIREMENTS \n', '2.2 WOOD PANEL PRODUCTS \n', '2.3 PRESERVATIVE-TREATED PLYWOOD \n', '2.4 FIRE-RETARDANT-TREATED PLYWOOD \n', '2.5 WALL SHEATHING \n', '2.6 ROOF SHEATHING \n', '2.7 PARAPET SHEATHING \n', '2.8 COMPOSITE NAIL BASE INSULATED ROOF SHEATHING \n', '2.9 SUBFLOORING AND UNDERLAYMENT \n', '2.10 FASTENERS \n', '2.11 SHEATHING JOINT-AND-PENETRATION TREATMENT MATERIALS \n', '2.12 MISCELLANEOUS MATERIALS \n']


In [0]:
index = 0
for category in categories:
    if "WALL SHEATHING" in category:
        print("Process all materials under WALL SHEATHING")
        index = categories.index(category)
        str_ws = text[text.find(categories[index]):text.find(categories[index+1])]
        #print(str_ws)
        p_material = re.compile(r"[A-Z][.][a-zA-Z, -]*[:]")
        materials = re.findall(p_material, str_ws)
        #print(materials)
        for index_m in range(len(materials)-1):
            m_name = ''
            m_type = ''
            m_location=''
            m_exposure=''
            m_level=''

            m_name = materials[index_m].split(".")[-1][:-1].strip()
            #print(str_ws.find(materials[index_m]), str_ws.find(materials[index_m+1]))
            str_material = str_ws[str_ws.find(materials[index_m]):str_ws.find(materials[index_m+1])]
            #print(m_name)
            #.find(categories[index+1])]
            #print(str_material)
            m_types = re.findall('DOC PS [1-2]| Structural [VI+VX]|Regular|Type V?[I+V?X]', str_material)
            print(m_types)
            for m_type in set(m_types):
                m = re.search('Exterior|Interior', str_material)
                if m:
                    m_location = m.group(0)
                m = re.search('Exposure [1-3]', str_material)
                if m:
                    m_exposure = m.group(0)
                m = re.search('Level [1-9]', str_material)
                if m:
                    m_level = m.group(0)
            #m = re.search('Exterior|Interior', str_material)
            #if m:
               # m_location = m.group(0)
                print("Material Name: ", m_name)
                print("Material Type: ", m_type)
                print("Material Location: ", m_location)
                print("Material Exposure: ", m_exposure)
                print("Material Level: ", m_level)
                print("\n")

Process all materials under WALL SHEATHING
['DOC PS 1', 'DOC PS 1', 'DOC PS 2', ' Structural I', ' Structural I']
Material Name:  Plywood Sheathing
Material Type:  DOC PS 1
Material Location:  Exterior
Material Exposure:  Exposure 1
Material Level:  


Material Name:  Plywood Sheathing
Material Type:   Structural I
Material Location:  Exterior
Material Exposure:  Exposure 1
Material Level:  


Material Name:  Plywood Sheathing
Material Type:  DOC PS 2
Material Location:  Exterior
Material Exposure:  Exposure 1
Material Level:  


['DOC PS 2', ' Structural I']
Material Name:  Oriented-Strand-Board Sheathing
Material Type:   Structural I
Material Location:  
Material Exposure:  Exposure 1
Material Level:  


Material Name:  Oriented-Strand-Board Sheathing
Material Type:  DOC PS 2
Material Location:  
Material Exposure:  Exposure 1
Material Level:  


['Type X', 'Regular', 'Type X']
Material Name:  Paper-Surfaced Gypsum Sheathing
Material Type:  Regular
Material Location:  
Material Expos

In [0]:
index = 0
for category in categories:
    if "SUBFLOORING AND UNDERLAYMENT" in category:
        #print("Process all materials under SUBFLOORING AND UNDERLAYMENT")
        index = categories.index(category)
        str_ws = text[text.find(categories[index]):text.find(categories[index+1])]
        #print(str_ws) #text 
        p_material = re.compile(r"[A-Z][.][a-zA-Z, -]*[:]")
        materials = re.findall(p_material, str_ws) #list of lettered subheadings
        for index_m in range(len(materials)-1): #len = 5, range = 4 ,so there are 5 lettered subheadings
            m_name = ''
            m_type = ''   
            m_location=''
            m_exposure=''
            m_level=''

            m_name = materials[index_m].split(".")[-1][:-1].strip() #take away the letter and the period in the subheading 
            #print(m_name)
            #print(str_ws.find(materials[index_m]), str_ws.find(materials[index_m+1])) #lowest index of when the subheading appears and the index of the next subheading
            str_material = str_ws[str_ws.find(materials[index_m]):str_ws.find(materials[index_m+1])]
            #.find(categories[index+1])]
            #print(str_material)
            m_types = re.findall('DOC PS [1-2]|Regular|Type V?[I+V?X]|C-C Plugged|Exposure 1, ?|Exposure 1\\[? |Exposure 1\\[|Structural I|Exterior', str_material)
            print(materials[index_m])
            #print(m_types)
            m_types2 = []
            for i in m_types:
              clean_i = i.strip().strip(",")
              if "[" in i:
                clean_i = i.replace("[","")
              if clean_i not in m_types2:
                m_types2.append(clean_i)
            print(m_types2)
            for m_type in set(m_types2):
                m = re.search('Exterior|Interior', str_material)
                if m:
                    m_location = m.group(0)
                    #print("m_location:",m_location)
                m = re.search('Exposure [1-3]', str_material)
                if m2:
                    m_exposure = m2.group(0)
                m = re.search('Level [1-9]', str_material)
                if m:
                    m_level = m.group(0)
            #m = re.search('Exterior|Interior', str_material)
            #if m:
               # m_location = m.group(0)
                print("Material Name: ", m_name)
                print("Material Type: ", m_type)
                print("Material Location: ", m_location)
                print("Material Exposure: ", m_exposure)
                print("Material Level: ", m_level)
                print("\n")

A. Plywood Combination Subfloor-Underlayment:
['DOC PS 1', 'Exterior', 'Structural I', 'C-C Plugged', 'Exposure 1']
Material Name:  Plywood Combination Subfloor-Underlayment
Material Type:  DOC PS 1
Material Location:  Exterior
Material Exposure:  
Material Level:  Exposure 1


Material Name:  Plywood Combination Subfloor-Underlayment
Material Type:  Exterior
Material Location:  Exterior
Material Exposure:  
Material Level:  Exposure 1


Material Name:  Plywood Combination Subfloor-Underlayment
Material Type:  Exposure 1
Material Location:  Exterior
Material Exposure:  
Material Level:  Exposure 1


Material Name:  Plywood Combination Subfloor-Underlayment
Material Type:  C-C Plugged
Material Location:  Exterior
Material Exposure:  
Material Level:  Exposure 1


Material Name:  Plywood Combination Subfloor-Underlayment
Material Type:  Structural I
Material Location:  Exterior
Material Exposure:  
Material Level:  Exposure 1


B. Oriented-Strand-Board Combination Subfloor-Underlayment:
